# ⚠️ Hot Jupiter Phase Curve Example

In [ ]:
%matplotlib inline
import warnings
warnings.simplefilter("ignore")

In [ ]:
import starry
import matplotlib.pyplot as plt
import numpy as np
import exoplanet as xo
import pymc3 as pm
import theano.tensor as tt
import theano

starry.config.quiet = True

In [ ]:
A = starry.Primary(
    starry.Map(ydeg=0, udeg=2, L=1.0, inc=90.0, obl=0.0),
    m=1.0,
    r=1.0,
    prot=1.0
)
A.map[1] = 0.4
A.map[2] = 0.2
A.map.show()

In [ ]:
b = starry.Secondary(
    starry.Map(ydeg=1, udeg=0, L=0.001, inc=90.0, obl=0.0),
    m=0.0,
    r=0.1,
    prot=1.0,
    porb=1.0
)
b.map[1, 0] = 0.5
offset = 10.0
b.theta0 = 180.0 + offset

In [ ]:
b.map.show()

In [ ]:
sys = starry.System(A, b)

In [ ]:
t = np.linspace(-0.3, 1.3, 1000)
flux_true = sys.flux(t).eval()
ferr = 1e-4
flux = flux_true + ferr * np.random.randn(len(t))
plt.plot(t, flux, "k.", alpha=0.3, ms=3)
plt.plot(t, flux_true);

In [ ]:
with pm.Model() as model:
    
    offset = pm.Normal("offset", 0.0, 30.0, testval=0.11)
    log_L = pm.Normal("log_L", -4.0, 2.0, testval=-3.91)
    
    A = starry.Primary(
        starry.Map(ydeg=0, udeg=2, L=1.0, inc=90.0, obl=0.0),
        m=1.0,
        r=1.0,
        prot=1.0
    )
    A.map[1] = 0.4
    A.map[2] = 0.2

    b = starry.Secondary(
        starry.Map(ydeg=1, udeg=0, L=10 ** log_L, inc=90.0, obl=0.0),
        m=tt.as_tensor_variable(0.0).astype("float64"),
        r=0.1,
        prot=1.0,
        porb=1.0
    )
    b.map[1, 0] = 0.5
    b.theta0 = 180.0 + offset
    
    sys = starry.System(A, b)
    
    flux_model = pm.Deterministic("flux_model", sys.flux(t))
    
    pm.Normal("obs", flux_model, sd=ferr, observed=flux)

In [ ]:
%%time
with model:
    map_soln = xo.optimize()

In [ ]:
plt.plot(t, flux, "k.", alpha=0.3, ms=3)
plt.plot(t, map_soln["flux_model"]);

In [ ]:
%%time
with model:
    trace = pm.sample(
        tune=250,
        draws=500,
        start=map_soln,
        chains=4,
        cores=1,
        step=xo.get_dense_nuts_step(target_accept=0.9),
    )

In [ ]:
pm.summary(trace, varnames=["log_L", "offset"])

In [ ]:
import corner
samples = pm.trace_to_dataframe(trace, varnames=["log_L", "offset"])
corner.corner(np.array(samples), truths=[-3, 10]);